<a href="https://colab.research.google.com/github/ShutterStack/Semantic-Chunking_EDA-Audio-Text/blob/main/gradio_setics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytube moviepy

from pytube import YouTube
from moviepy.editor import VideoFileClip

# Download the video from YouTube
def download_video(url, output_path='video.mp4'):
    yt = YouTube(url)
    yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first().download(filename=output_path)
    return output_path

# Extract audio from the downloaded video
def extract_audio(video_path, output_path='audio.wav'):
    video = VideoFileClip(video_path)
    video.audio.write_audiofile(output_path)
    return output_path

video_url = 'https://www.youtube.com/watch?v=Sby1uJ_NFIY'
video_path = download_video(video_url)
audio_path = extract_audio(video_path)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.7 MB/s eta 0:00:00
MoviePy - Writing audio in audio.wav


MoviePy - Done.


In [2]:
!pip install openai-whisper

import whisper

# Load the Whisper model
model = whisper.load_model("base")

# Transcribe the audio
def transcribe_audio(audio_path):
    result = model.transcribe(audio_path)
    return result['segments']

segments = transcribe_audio(audio_path)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using ca

100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 102MiB/s]
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



In [3]:
import re
import whisper

# Load the Whisper model
model = whisper.load_model("base")

# Transcribe the audio
def transcribe_audio(audio_path):
    result = model.transcribe(audio_path)
    return result['segments']

# Function to refine the time-aligned transcripts
def refine_segments(segments, max_chunk_length=15.0, min_chunk_length=5.0, overlap=0.5):
    refined_segments = []
    current_segment = []
    current_start = None

    for segment in segments:
        start, end, text = segment['start'], segment['end'], segment['text']

        if current_start is None:
            current_start = start

        # Add segment to current chunk
        current_segment.append(segment)
        current_chunk_length = end - current_start

        # Check if the current chunk exceeds max_chunk_length or ends with a sentence
        if current_chunk_length > max_chunk_length or re.search(r'[.!?]', text.strip()):
            # Create a new chunk
            chunk_text = " ".join([seg['text'] for seg in current_segment])
            refined_segments.append({
                "start_time": current_start,
                "end_time": end,
                "text": chunk_text,
            })
            # Start a new segment
            current_segment = []
            current_start = end + overlap

    # Add the last segment if any
    if current_segment:
        chunk_text = " ".join([seg['text'] for seg in current_segment])
        refined_segments.append({
            "start_time": current_start,
            "end_time": end,
            "text": chunk_text,
        })

    # Merge short segments with adjacent ones
    final_segments = []
    current_segment = []
    current_start = None

    for segment in refined_segments:
        start, end, text = segment['start_time'], segment['end_time'], segment['text']

        if current_start is None:
            current_start = start

        # Add segment to current chunk
        current_segment.append(segment)
        current_chunk_length = end - current_start

        # Check if the current chunk is less than min_chunk_length
        if current_chunk_length < min_chunk_length and len(final_segments) > 0:
            # Merge with previous segment
            final_segments[-1]['end_time'] = end
            final_segments[-1]['text'] += " " + text
        else:
            # Create a new chunk
            final_segments.append({
                "start_time": current_start,
                "end_time": end,
                "text": text,
            })
            current_segment = []
            current_start = end

    return final_segments

# Example Usage
segments = transcribe_audio(audio_path)
refined_segments = refine_segments(segments)

# Display the refined segments
for segment in refined_segments:
    print(segment)


  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



KeyboardInterrupt: 

In [ ]:
import re

# Define a function to perform semantic chunking
def semantic_chunking(segments, max_chunk_length=15.0):
    chunks = []
    chunk_id = 0
    current_chunk = []
    current_start = None

    for segment in segments:
        start, end, text = segment['start'], segment['end'], segment['text']

        if current_start is None:
            current_start = start

        # Check if adding the current segment exceeds max_chunk_length
        if end - current_start > max_chunk_length:
            # Create a new chunk
            chunk_text = " ".join([seg['text'] for seg in current_chunk])
            chunks.append({
                "chunk_id": chunk_id,
                "chunk_length": end - current_start,
                "text": chunk_text,
                "start_time": current_start,
                "end_time": end,
            })
            chunk_id += 1
            current_chunk = []
            current_start = start

        current_chunk.append(segment)

    # Add the last chunk
    if current_chunk:
        chunk_text = " ".join([seg['text'] for seg in current_chunk])
        chunks.append({
            "chunk_id": chunk_id,
            "chunk_length": end - current_start,
            "text": chunk_text,
            "start_time": current_start,
            "end_time": end,
        })

    return chunks

chunks = semantic_chunking(segments)

# Display the chunks
for chunk in chunks:
    print(chunk)


In [ ]:
!pip install gradio

import gradio as gr

def process_video(url):
    video_path = download_video(url)
    audio_path = extract_audio(video_path)
    segments = transcribe_audio(audio_path)
    chunks = semantic_chunking(segments)
    return chunks

iface = gr.Interface(
    fn=process_video,
    inputs="text",
    outputs="json",
    title="YouTube Video Semantic Chunker",
    description="Input a YouTube video URL to get semantically chunked audio-text pairs."
)

iface.launch()
